In [1]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
# get data
data = pd.read_csv('Level_3b_NonPath_SP2_EsCo_PSL_1_2_3_4_574.csv') 

# split data
X=data.iloc[:,51:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

     Schneider.Xr.A  Schneider.Xr.R  ...  Grantham.Xd.29  Grantham.Xd.30
0          0.069526        0.063206  ...        0.032370        0.030361
1          0.108020        0.038820  ...        0.032770        0.032138
2          0.099133        0.052175  ...        0.031862        0.027646
3          0.227360        0.028420  ...        0.030777        0.032884
4          0.085301        0.051630  ...        0.032222        0.031622
..              ...             ...  ...             ...             ...
569        0.132721        0.070508  ...        0.035352        0.034787
570        0.116286        0.037971  ...        0.031048        0.031582
571        0.091348        0.079168  ...        0.029097        0.031874
572        0.108571        0.056547  ...        0.030560        0.031774
573        0.091774        0.030591  ...        0.031451        0.031101

[574 rows x 100 columns]
0      1
1      1
2      1
3      1
4      1
      ..
569    4
570    4
571    4
572    4
573    4

In [3]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [4]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.2,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(100, 4 - 1) = 3 components.
  ChangedBehaviorWarn

In [5]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 100)            40800     
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            117248    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 4)                 260       
Total params: 207,716
Trainable params: 207,716
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/1000
30/30 [==============================] - 36s 108ms/step - loss: 0.6707 - accuracy: 0.3949 - val_loss: 0.5303 - val_accuracy: 0.4831
Epoch 2/1000
30/30 [==============================] - 1s 17ms/step - loss: 0.4900 - accuracy: 0.5035 - val_loss: 0.4225 - val_accuracy: 0.5720
Epoch 3/1000
30/30 [==============================] - 1s 17ms/step - loss: 0.3961 - accuracy: 0.6136 - val_loss: 0.3286 - val_accuracy: 0.7627
Epoch 4/1000
30/30 [==============================] - 1s 17ms/step - loss: 0.3499 - accuracy: 0.6905 - val_loss: 0.2944 - val_accuracy: 0.7712
Epoch 5/1000
30/30 [==============================] - 1s 17ms/step - loss: 0.3096 - accuracy: 0.7290 - val_loss: 0.2672 - val_accuracy: 0.8051
Epoch 6/1000
30/30 [==============================] - 0s 16ms/step - loss: 0.2711 - accuracy: 0.7826 - val_loss: 0.2325 - val_accuracy: 0.8390
Epoch 7/1000
30/30 [==============================] - 0s 16ms/step - loss: 0.2390 - accuracy: 0.8307 - val_loss: 0.1907 - val_accuracy: 0.86

In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.9170212765957447
test-acc = 0.847457627118644
train-CFR =               precision    recall  f1-score   support

           0       0.90      0.92      0.91       230
           1       0.91      0.94      0.93       241
           2       0.93      0.91      0.92       227
           3       0.92      0.90      0.91       242

    accuracy                           0.92       940
   macro avg       0.92      0.92      0.92       940
weighted avg       0.92      0.92      0.92       940

test-CFR =               precision    recall  f1-score   support

           0       0.87      0.70      0.78        64
           1       0.75      0.87      0.81        53
           2       0.93      0.99      0.96        67
           3       0.83      0.83      0.83        52

    accuracy                           0.85       236
   macro avg       0.84      0.85      0.84       236
weighted avg       0.85      0.85      0.85       236

